In [2]:
from osgeo import gdal, ogr

In [3]:
input_raster = gdal.Open(r"data\original\bathymetry_hawaii.tif")
type(input_raster)

osgeo.gdal.Dataset

In [13]:
# Projection
input_raster.GetProjection()

'GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Latitude",NORTH],AXIS["Longitude",EAST],AUTHORITY["EPSG","4326"]]'

In [8]:
input_shape_3mile = "https://opendata.arcgis.com/datasets/82fb905c78064559905c8d932b63478b_31.geojson"
output_raster_3mile = "data\processed/output_3mile.tif"

ds = gdal.Warp(output_raster_3mile,
              input_raster,
              cropToCutline=True,
              cutlineDSName = input_shape_3mile, # or any other file format
              format = 'GTiff',
              dstNodata = -9999) # select the no data value you like
ds=None #do other stuff with ds object, it is your cropped dataset. in this case we only close the dataset. 

In [16]:
input_shape_coastline = "https://opendata.arcgis.com/datasets/045b1d5147634e2380566668e04094c6_3.geojson"
output_raster_coastline = "data\processed/output_coastline.tif"

gdal.Rasterize(output_raster_coastline, input_shape_coastline, burn_values=[1])

TypeError: RasterizeOptions() got an unexpected keyword argument 'burn_values'